# import các thư viện

In [1]:
import cv2
import torch
import numpy as np
from skimage import measure
import os
import pandas as pd


c:\Users\dangt\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# dùng model là yolov5

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='weights/best.pt')

Using cache found in C:\Users\dangt/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-3-20 Python-3.10.0 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


# hàm xuất ra ma trận nhị phân đặc trưng cho kí tự

In [3]:
# Xuất ra ma trận nhị phân ( đen = 0, trắng = 255)
def feature(folder_path, file):
    binaries = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            # Đọc tệp tin ảnh
            img = cv2.imread(os.path.join(folder_path, filename), cv2.IMREAD_GRAYSCALE)

            # Đổi ảnh sang ma trận nhị phân
            ret, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

            # Thêm ma trận nhị phân vào mảng
            binaries.append(binary)
        

    # Khởi tạo mảng pixel trắng đen có kích thước giống với các ma trận trong binaries với các phần tử được khởi tạo là 0
    black_pixels = np.zeros_like(binaries[0])
    white_pixels = np.zeros_like(binaries[0])
    
    # Tính toán số lượng pixel màu đen và màu trắng cho từng pixel trong các ma trận
    for binary in binaries:
        black_pixels += binary == 0
        white_pixels += binary == 255

    # So sánh số lượng pixel màu đen và màu trắng để quyết định màu sắc cuối cùng của pixel trong ma trận đặc trưng
    feature = np.zeros_like(binaries[0], dtype=np.uint8)
    for i in range(feature.shape[0]):
        for j in range(feature.shape[1]):
            if black_pixels[i][j] > white_pixels[i][j]:
                feature[i][j] = 0
            else:
                feature[i][j] = 255

    # xuất ra ma trận nhị phân đặc trưng, lưu vào tệp trong folder featured
    np.savetxt('featured/featured'+file+'.txt', feature, fmt='%d')
    return feature

# xuất ra ma trận đặc trưng cho tất cả các kí tự

In [4]:
features = []
folder_path = "char/"
files = os.listdir(folder_path)
for file in files:
    if os.path.isdir(os.path.join(folder_path, file)):
        folder_path_ii = os.path.join(folder_path, file)
        features.append(feature(folder_path_ii, file))

# so sánh để đưa ra kết luận

In [5]:
def predict(binary, number):
    n = 0
    max = 0
    i = 0

    # number == 1: kí tự đang đọc là kí tự chữ số
    if (number == 1):

        # Đọc từ 0 đến 9
        for i in range (10):

            # Tính số pixel giống nhau của chữ số cần dự đoán và đặc trưng của các chữ số
            matching_pixels = np.sum(binary == features[i])
            total_pixels = binary.shape[0] * binary.shape[1]

            # Tính % giống nhau bằng cách lấy số pixel giống nhau / tổng số pixel * 100
            matching_percentage = matching_pixels / total_pixels * 100

            # Lấy max = tỉ lệ khớp nhất của chữ số cần dự đoán với các chữ số đặc trưng, n là chữ số được dự đoán
            if (matching_percentage>max): 
                n = i
                max = matching_percentage

    # number == 0: kí tự đang đọc là kí tự chữ cái
    else:

        # Nếu không phải chữ số thì đọc các feature chữ cái
        for i in range(10, len(features)):
            matching_pixels = np.sum(binary == features[i])
            total_pixels = binary.shape[0] * binary.shape[1]
            matching_percentage = matching_pixels / total_pixels * 100
            if (matching_percentage>max): 
                n = i
                max = matching_percentage
    # print(n,max)
    if (n == 7 and max < 65): 
        n = 1
    return n, max

# đọc ký tự trên biển số xe

In [13]:
def readPlate(image, model):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model(image)

    for i, det in enumerate(results.xyxy[0]):
        # Lấy tọa độ bbox của vật thể thứ i
        bbox = det[0:4].cpu().numpy()

        # Cắt lấy vùng ảnh nằm trong bbox
        cropped_image = image[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
        cropped_image = cv2.resize(cropped_image, (256, 100))
        # Chuyển ảnh sang đen trắng
        gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
        # blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        # edges = cv2.Canny(blurred, threshold1=30, threshold2=100)
        # edges = cv2.bitwise_not(edges)
        
        # Sử dụng hàm connectedComponents để tìm các vùng kết nối trên ảnh nhị phân
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        num_labels, labels = cv2.connectedComponents(binary)
        # cv2.imshow("Binary",binary)
        # cv2.waitKey()
        # cv2.destroyAllWindows()

        # Hiển thị số lượng các labels tìm được

        d = 0
        # Sử dụng hàm connectedComponentsWithStats để tính toán diện tích của các vùng kết nối
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary)
        for i in range(num_labels):
            area = stats[i, cv2.CC_STAT_AREA]
            if area < 100 or area > 1500:
                labels[labels == i] = 0
            else: 
                d = d+1
        
        x_centroids = centroids[:, 0] # Lấy tất cả các hàng, cột đầu tiên (tương ứng với tọa độ x)
        
        xx = []
        # Lặp qua các nhãn
        if (d > 5):
            for i in range(1, num_labels):
            # Lấy thông tin của nhãn hiện tại
                x, y, w, h, area = stats[i]

            # Nếu diện tích của nhãn nằm trong khoảng từ 100 đến 1500
                if 100 < area < 1500:
                # Cắt ảnh của nhãn hiện tại từ ảnh gốc
                    digit_img = binary[y:y+h, x:x+w]
                    digit_img = cv2.resize(digit_img, (25,60))
                    xx.append((x_centroids[i], digit_img))         
        else:
            return "0"

        sorted_list = sorted(xx, key=lambda x: x[0])

        predicted = []
        delete = 0
        if (len(sorted_list) > 8):
            # print(len(sorted_list))
            delete = len(sorted_list) - 8
            # # Sắp xếp danh sách predicted theo độ chính xác giảm dần

        # print(delete)
        num = []
        for i in range(len(sorted_list)):
            x, value = sorted_list[i]
            predicted.append((i, predict(value, 1), predict(value, 0)))
        
        predicted = sorted(predicted, key=lambda x: x[1][1], reverse=True)
        predicted_string = ""
        # print(predicted)
        j = 0
        lst = [list(item) for item in predicted]
        # print(delete)
        # check = 0
        for i in reversed(range(len(predicted))):
            # print(predicted[i])
            # print(lst[i])
            if (j == delete):
                break
            if (lst[i][0] < 2):
                lst.remove(lst[i])
                # for k in range(len(lst)):
                #     if (lst[k][0] >= 2):
                #         lst[k][0] = lst[k][0] - 1
                j = j + 1
            elif (lst[i][0] > (2+ delete)):
                lst.remove(lst[i])
                j = j + 1
            else:
                # print("123")
                if (lst[i][2][1] >= 65):
                    lst.remove(lst[i-1])
                    j = j + 1
                else:
                    lst.remove(lst[i])
                    j = j + 1
        # print(lst)

        predicted = [tuple(item) for item in lst] # chuyển lại thành tuple
        i = 0
        for item in sorted(lst, key=lambda x: x[0]):
            i = i + 1
            # print(item)
            if (i != 3):
                predicted_string += str(item[1][0])
            else:
                if (item[2][0] == 10):
                    predicted_string += "A"
                if (item[2][0] == 11):
                    predicted_string += "B"
                if (item[2][0] == 12):
                    predicted_string += "C"
                if (item[2][0] == 13):
                    predicted_string += "D"
                if (item[2][0] == 14):
                    predicted_string += "F"
                if (item[2][0] == 15):
                    predicted_string += "G"
                if (item[2][0] == 16):
                    predicted_string += "L"
                if (item[2][0] == 17):
                    predicted_string += "N"
                if (item[2][0] == 18):
                    predicted_string += "S"
                if (item[2][0] == 19):
                    predicted_string += "V"
                if (item[2][0] == 20):
                    predicted_string += "Y"
                if (item[2][0] == 21):
                    predicted_string += "Z"
        # print(predicted_string)
        return predicted_string

# Test những biển số sai sót

In [14]:
img = cv2.imread("car_long/782.jpg") # đọc ảnh bằng OpenCV
readPlate(img, model)

'30V39931'

# xuất ra file csv kết quả kiểm thử hình ảnh trong tập kiểm thử

In [15]:
folder_path = "car_long/" # đường dẫn đến folder chứa ảnh

pre_final = []
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"): 
        # kiểm tra xem file có phải là ảnh jpg hoặc png không
        file_path = os.path.join(folder_path, filename)
        img = cv2.imread(file_path) # đọc ảnh bằng OpenCV
        # sử dụng ảnh tại đây
        name, extension = os.path.splitext(filename) # Tách phần đuôi của tên tệp
        pre_final.append((name, readPlate(img, model)))

# Tạo DataFrame từ mảng
df = pd.DataFrame(pre_final, columns=['Image', 'Predicted'])

# Xuất ra file CSV
df.to_csv('predicted_output.csv', index=False)


# đọc video kiểm thử 

In [9]:
# # Tạo một đối tượng VideoCapture để đọc video
# cap = cv2.VideoCapture('test.MOV')

# # Lấy kích thước khung hình của video
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# # Tạo đối tượng VideoWriter để ghi video
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter('processed_video.mp4', fourcc, 30, (width, height))
# # Đọc từng khung hình của video, xử lý và ghi lại
# result_s = ''
# read = 0

# with open("license_plates_history.txt", "w") as file:
#     file.write('')

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         break
#     # Xử lý khung hình ở đây
#     processed_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = model(processed_frame)
    
#     if len(results.pred[0]) >= 1:
#         if (results.pred[0][0][0].item() <= 0.5):
#             read = 0
#         for i in range (len(results.pred[0])):
#             accuracy = float(results.pred[0][i][4].item())
#             if (accuracy >= 0.95 and results.pred[0][i][0].item() <= width/2 and read == 0):
#                 if (len(readPlate(processed_frame,model)) >= 7 and len(readPlate(processed_frame,model)) <= 8):
#                     result_s = readPlate(processed_frame,model)
#                     read = 1
#                     with open('license_plates_history.txt', 'a') as file:
#                         file.write(result_s + '\n')
#     # Vẽ chuỗi result_s lên khung hình results
#     cv2.putText(results.render()[0], result_s, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#     # Ghi khung hình đã xử lý vào đối tượng VideoWriter
#     out.write(cv2.cvtColor(results.render()[0], cv2.COLOR_RGB2BGR))

#     # # Hiển thị khung hình đã xử lý
#     cv2.imshow('Processed Frame', cv2.cvtColor(results.render()[0], cv2.COLOR_RGB2BGR))
#     if cv2.waitKey(1) == ord('q'):
#         break

# # Giải phóng các tài nguyên và đóng các cửa sổ hiển thị
# out.release()
# cv2.destroyAllWindows()

In [16]:

# Đọc file CSV vào DataFrame
df = pd.read_csv('predicted_output.csv')

# Sắp xếp DataFrame theo cột số
df_sorted = df.sort_values(by='Image', ascending=True)

# Lưu lại kết quả vào file CSV
df_sorted.to_csv('predicted_output.csv', index=False)

In [17]:
# Đọc hai file CSV và lưu chúng vào hai DataFrame
df1 = pd.read_csv('predicted_output.csv')
df2 = pd.read_csv('true_output.csv')

# Kết hợp hai DataFrame và lấy các cột cần thiết
merged_df = pd.merge(df1[['Image', 'Predicted']], df2[['Image', 'True']], on='Image')

merged_df['Diff'] = merged_df.apply(lambda row: 0 if row['Predicted'] == row['True'] else 1, axis=1)

# Thêm cột Predict dựa trên giá trị của cột thứ 2
merged_df['Predict'] = merged_df['Predicted'].apply(lambda x: 0 if x == "0" else 1)

merged_df['Miss'] = merged_df.apply(lambda row: 1 if len(row['Predicted']) - len(row['True']) != 0 else 0, axis=1)

# Lưu các cột vào file CSV
merged_df.to_csv('Evaluate.csv', index=False)



In [19]:
# Đọc file CSV vào DataFrame
merged_df = pd.read_csv('Evaluate.csv')

zero_Predict_ratio = (merged_df['Predict'] == 0).sum() / len(merged_df) * 100
miss_Predict_ratio = (merged_df['Miss'] == 1).sum()/ len(merged_df) * 100
right_Predict_ratio = 100 - ((merged_df['Diff'] == 1).sum()) / len(merged_df) * 100
wrong_Predict = (merged_df['Diff'] == 1).sum()- (merged_df['Predict'] == 0 - (merged_df['Miss'] == 1).sum()).sum()
wrong_Predict_ratio = wrong_Predict / len(merged_df) * 100

print("Số ảnh kiểm thử: ", len(merged_df))
print("Tỉ lệ không thể nhận diện biển số: {:.2f}%".format(zero_Predict_ratio))
print("Tỉ lệ đọc sai biển số: {:.2f}%".format(wrong_Predict_ratio- miss_Predict_ratio))
print("Tỉ lệ đọc thiếu chữ số: {:.2f}%".format(miss_Predict_ratio))
print("Tỉ lệ biển số đọc đúng: {:.2f}%".format(right_Predict_ratio))

mask = (merged_df['Diff'] == 1)
df_filtered = merged_df.loc[mask]
print(df_filtered)

Số ảnh kiểm thử:  474
Tỉ lệ không thể nhận diện biển số: 1.27%
Tỉ lệ đọc sai biển số: 4.64%
Tỉ lệ đọc thiếu chữ số: 3.16%
Tỉ lệ biển số đọc đúng: 92.19%
     Image Predicted      True  Diff  Predict  Miss
30     509  51A89711  51A89714     1        1     0
34     554    54Y210  54A72110     1        1     1
35     555         0  54A72110     1        0     1
42     647  51F24483  51F24403     1        1     0
52     782  30V39931   30V3993     1        1     1
53     783  30V39931   30V3993     1        1     1
67    1961         0  51A85325     1        0     1
119   3820  51B21666  51D21666     1        1     0
124   3843  51A96141  51A91614     1        1     0
143   5347  51G25182  51G25181     1        1     0
183   6897    51Z210  51A72110     1        1     1
185   6905  51A19642  51A96141     1        1     0
197   7078  28A51796  29A51796     1        1     0
206   7214   15A5380  51F15380     1        1     1
207   7215   15A5381  51F15380     1        1     1
234   8814  51F